In [1]:
import pandas as pd

In [7]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

#### 각 날짜의 1시간 전의 기상상황을 가지고 1시간 후의 따릉이 대여수를 예측해보세요. 

[Data fields] <br>
① train.csv - 서울시 날짜에 따른 시간별 기상상황과 따릉이 대여 수 (기간: 2017년 4월, 5월)
 <br>
id : 날짜와 시간별 id <br>
temperature : 기온 <br>
precipitation : 비가 오지 않았으면 0, 비가 오면 1 <br>
windspeed : 풍속(평균) <br>
humidity : 습도 <br>
visibility : 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미) <br>
ozone : 오존 <br>
pm10 : 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지) <br>
pm2.5 : 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지) <br>
count : 시간에 따른 따릉이 대여 수 <br>
 <br>
③ submission.csv : submission 파일의 예시 <br>
id : 날짜와 시간별 id <br>
count : 예측한 대여 수 기입 <br> 

In [122]:
train = pd.read_csv('train_final.csv') # 판다스를 이용해 csv 파일을 읽어와 train 변수에 저장

test = pd.read_csv('test_final.csv') # 판다스를 이용해 csv 파일을 읽어와 train 변수에 저장

In [123]:
train.columns

Index(['id', 'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count'],
      dtype='object')

In [124]:
#train['temp_humid'] = train['hour_bef_temperature']/train['hour_bef_humidity']  #Feature Engineering 시도
train['visibility_pm10'] = train['hour_bef_visibility']/train['hour_bef_pm10']
#train['visibility_pm2.5'] = train['hour_bef_visibility']/train['hour_bef_pm2.5']

In [125]:
train.columns

Index(['id', 'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count',
       'visibility_pm10'],
      dtype='object')

In [126]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,visibility_pm10
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0,7.578947
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0,12.547945
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0,43.187500
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0,12.613333
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0,74.074074
5,13,2,13.6,0.0,1.7,80.0,1073.0,0.027,34.0,15.0,39.0,31.558824
6,14,3,10.6,0.0,1.5,58.0,1548.0,0.038,62.0,33.0,23.0,24.967742
7,16,21,16.0,0.0,6.0,21.0,1961.0,0.050,90.0,28.0,146.0,21.788889
8,19,9,13.8,0.0,1.9,64.0,1344.0,0.039,93.0,19.0,39.0,14.451613
9,20,14,17.2,0.0,2.1,32.0,1571.0,0.025,64.0,19.0,83.0,24.546875


In [127]:
input_var = ['id', 'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'visibility_pm10']

target = 'count'

In [129]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=1000, max_features=7)

In [130]:
rf.fit(train[input_var].fillna(0),train[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=7, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [131]:
#test['temp_humid'] = test['hour_bef_temperature']/test['hour_bef_humidity']
test['visibility_pm10'] = test['hour_bef_visibility']/test['hour_bef_pm10']
#test['visibility_pm2.5'] = test['hour_bef_visibility']/test['hour_bef_pm2.5']

In [132]:
prediction_rf=rf.predict_proba(test[input_var].fillna(0))[:,1]
#랜덤포레스트 데이터를 이용하여 test데이터를 예측.
#학습은 랜덤포레스트로 시킨 것이다.
import xgboost as xgb
xgb_param={'objective':'reg:linear',
          'metric':'rmse',
          'max_depth':6,
          'eta':0.03,
          'min_child_samples':100}
#xgboost로 학습하기 위해.

In [133]:
from sklearn.model_selection import KFold
folds =KFold(n_splits=5,shuffle=True)

In [134]:
prediction_xgb=np.zeros(len(test))
#xgboost로 학습.
for fold_, (trn_idx,val_idx) in enumerate(folds.split(train[input_var].values,train[target].values)):
    trn_data=xgb.DMatrix(train[input_var].iloc[trn_idx],train[target].iloc[trn_idx])
    val_data=xgb.DMatrix(train[input_var].iloc[val_idx],train[target].iloc[val_idx])
    
    num_tree=10000
    
    xgb_model=xgb.train(xgb_param,
                     trn_data,
                     num_tree,
                     [(trn_data,'train'),(val_data,'valid')],verbose_eval=0,
                     early_stopping_rounds=100)
    prediction_xgb+=xgb_model.predict(xgb.DMatrix(test[input_var]),ntree_limit=xgb_model.best_ntree_limit)

prediction_xgb=prediction_xgb/5

[23:26:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:26:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:26:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:26:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:26:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [135]:
final=pd.DataFrame({'pred_xgb':prediction_xgb,'pred_rf':prediction_rf})
final.head()

,pred_xgb,pred_rf
0,99.440785,0.000
1,203.412793,0.000
2,69.734885,0.044
3,41.279790,0.000
4,65.721683,0.071


In [136]:
final['pred_final'] = final['pred_xgb']/2+final['pred_rf']/2

In [137]:
pd.DataFrame({'id':test['id'], 'count':final['pred_xgb']}).to_csv("submission.csv", index = False)

In [138]:
final.head()

,pred_xgb,pred_rf,pred_final
0,99.440785,0.000,49.720393
1,203.412793,0.000,101.706396
2,69.734885,0.044,34.889442
3,41.279790,0.000,20.639895
4,65.721683,0.071,32.896341


In [139]:
folds=KFold(n_splits=5,shuffle=True)

In [140]:
oof_rf=np.zeros(len(train))

prediction_rf=np.zeros(len(test))

for fold_, (trn_idx,val_idx) in enumerate(folds.split(train[input_var].values,train[target].values)):
    rf.fit(train[input_var].iloc[trn_idx].fillna(0),train[target].iloc[trn_idx])
    oof_rf[val_idx]=rf.predict_proba(train[input_var].fillna(0).iloc[val_idx])[:,1]
    prediction_rf+=rf.predict_proba(test[input_var].fillna(0))[:,1]

prediction_rf=prediction_rf/5

In [141]:
prediction_rf

array([0.0004, 0.    , 0.0486, 0.    , 0.058 , 0.    , 0.0004, 0.    ,
       0.0002, 0.0194, 0.    , 0.    , 0.0002, 0.    , 0.0002, 0.0002,
       0.001 , 0.0158, 0.    , 0.    , 0.    , 0.0022, 0.002 , 0.0004,
       0.    , 0.0002, 0.0368, 0.    , 0.    , 0.001 , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.0004, 0.0006, 0.012 , 0.    , 0.001 ,
       0.0004, 0.    , 0.    , 0.0038, 0.    , 0.    , 0.0132, 0.    ,
       0.0006, 0.    , 0.    , 0.    , 0.0012, 0.0152, 0.    , 0.    ,
       0.0004, 0.    , 0.0252, 0.0036, 0.0006, 0.0006, 0.0016, 0.    ,
       0.    , 0.    , 0.0024, 0.0002, 0.001 , 0.    , 0.0002, 0.0002,
       0.0004, 0.002 , 0.    , 0.    , 0.0174, 0.0004, 0.    , 0.0024,
       0.0002, 0.    , 0.0034, 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.0022, 0.0004, 0.    , 0.0014, 0.    , 0.    ,
       0.0014, 0.003 , 0.    , 0.    , 0.0152, 0.0002, 0.0034, 0.0002,
       0.0012, 0.0544, 0.0002, 0.002 , 0.0002, 0.017 , 0.    , 0.    ,
      

In [142]:
oof_xgb=np.zeros(len(train))

prediction_xgb=np.zeros(len(test))

for fold_, (trn_idx,val_idx) in enumerate(folds.split(train[input_var].values,train[target].values)):
    trn_data=xgb.DMatrix(train[input_var].iloc[trn_idx],train[target].iloc[trn_idx])
    val_data=xgb.DMatrix(train[input_var].iloc[val_idx],train[target].iloc[val_idx])
    num_tree=10000
    xgb_model=xgb.train(xgb_param,
                     trn_data,
                     num_tree,
                     [(trn_data,'train'),(val_data,'valid')],verbose_eval=0,
                     early_stopping_rounds=100)
    oof_xgb[val_idx]=xgb_model.predict(xgb.DMatrix(train[input_var].iloc[val_idx]),ntree_limit=xgb_model.best_ntree_limit)
    prediction_xgb+=xgb_model.predict(xgb.DMatrix(test[input_var]),ntree_limit=xgb_model.best_ntree_limit)
prediction_xgb=prediction_xgb/5

[23:31:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:31:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:32:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:32:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:32:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [143]:
oof_rf

array([0.023, 0.   , 0.004, ..., 0.   , 0.   , 0.   ])

In [144]:
oof_xgb

array([ 65.1027832 , 120.86112976,  13.84511948, ...,  23.35144043,
       232.48722839, 188.64613342])

In [145]:
pd.DataFrame({'oof_rf':oof_rf, 'oof_xgb':oof_xgb, 'count':train['count']})

,oof_rf,oof_xgb,count
0,0.023,65.102783,49.0
1,0.000,120.861130,159.0
2,0.004,13.845119,26.0
3,0.002,63.722900,57.0
4,0.000,227.521759,431.0
5,0.000,31.908369,39.0
6,0.000,21.061361,23.0
7,0.000,114.187576,146.0
8,0.004,60.719715,39.0
9,0.001,123.818680,83.0


In [146]:
train_super=pd.DataFrame({'oof_rf':oof_rf,'oof_xgb':oof_xgb,'count':train['count']})
test_super=pd.DataFrame({'pred_rf':prediction_rf,'pred_xgb':prediction_xgb})

In [147]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_super[['oof_rf','oof_xgb']], train_super['count'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [148]:
lr.predict(test_super[['pred_rf', 'pred_xgb']])

array([101.69999051, 199.74041602,  64.73964166,  42.78515742,
        55.08333712, 132.80216582, 243.26002176, 236.44309729,
        37.81400467, 139.14983708, 287.22684542, 234.24149301,
        96.40082651,  49.71957516, 180.93929528, 153.35227462,
        28.9531809 , 173.20317729, 342.16564659, 170.13379917,
       230.0416603 ,  87.18049499,  22.9597856 , 136.86152871,
       131.31150329, 104.19154833,  25.45349696, 118.20693451,
       118.15174671, 151.6119118 ,  70.99644237,  35.00801636,
        69.69972767, 116.98852129, 273.79103745,  35.20055638,
       127.46584611, 137.20566413, 209.26086509,  74.86508577,
        48.24682916, 116.86388753, 175.82794813,  72.40009785,
       303.6051874 , 194.07414496,  95.0135935 ,  64.03028921,
        26.4357254 ,  90.38953682, 242.38064805,  97.67558215,
       107.58005637,  85.49751533, 197.93237231, 158.21107355,
        52.09126893, 181.41338499,  24.65839892,  25.30788511,
        93.94254256,  85.95946596, 269.58474323, 263.21

In [149]:
final_predictions = lr.predict(test_super[['pred_rf', 'pred_xgb']])

In [150]:
pd.DataFrame({'id':test['id'], 'count':final_predictions}).to_csv("submission.csv",index=False)
